In [15]:
from functools import partial
import os
import tempfile
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from ray import tune
from ray import train
from ray.train import Checkpoint, get_checkpoint
from ray.tune.schedulers import ASHAScheduler
import ray.cloudpickle as pickle
from ultralytics import YOLO

In [16]:
TASK2METRIC = {
    "detect": "metrics/mAP50(B)", #detect 지표를 mAP50-95에서 50으로 변경하기 위함
    "segment": "metrics/mAP50-95(M)",
    "classify": "metrics/accuracy_top1",
    "pose": "metrics/mAP50-95(P)",
    "obb": "metrics/mAP50-95(B)",
}

In [17]:
experiment_name = 'tune'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# Load a YOLOv8n model
best_model_tuning = YOLO("yolov8n.pt")

task = best_model_tuning.task
metric = TASK2METRIC[task] 

result_grid = best_model_tuning.tune(
    data="/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml",
    epochs=200,
    use_ray=True,
    name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
    gpu_per_trial=1  # 각 트라이얼에 1개의 GPU 할당
)

(_tune pid=16924) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=16926) engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=tune_raytune, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, 

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
  7%|▋         | 472k/6.25M [00:00<00:01, 4.79MB/s]
 59%|█████▊    | 3.66M/6.25M [00:01<00:00, 3.23MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 4.76MB/s]
 95%|█████████▌| 5.95M/6.25M [00:01<00:00, 4.03MB/s]
100%|██████████| 6.25M/6.25M [00:01<00:00, 3.50MB/s]
 95%|█████████▍| 5.93M/6.25M [00:02<00:00, 5.53MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.94MB/s]
100%|██████████| 6.25M/6.25M [00:02<00:00, 2.99MB/s]


(_tune pid=16923) AMP: checks passed ✅
(_tune pid=16924) Overriding model.yaml nc=80 with nc=2 [repeated 3x across cluster]
(_tune pid=16924)                    from  n    params  module                                       arguments                      [repeated 3x across cluster]
(_tune pid=16924)  19                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]               [repeated 21x across cluster]
(_tune pid=16924)   8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]            [repeated 12x across cluster]
(_tune pid=16924)   9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                  [repeated 3x across cluster]
(_tune pid=16924)  13                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None, 2, 'nearest']           [repeated 6x across cluster]
(_tune pid=16924)  20             [-1, 9]  1         0  ultralytics.n

train: Scanning /home/under1/Detect/dataset/coronal/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
  0%|          | 0.00/6.25M [00:00<?, ?B/s] [repeated 3x across cluster]
 91%|█████████ | 5.68M/6.25M [00:02<00:00, 4.38MB/s] [repeated 60x across cluster]
 74%|███████▍  | 4.61M/6.25M [00:01<00:00, 4.14MB/s] [repeated 2x across cluster]


(_tune pid=16926) Plotting labels to runs/detect/tune/tune_raytune/labels.jpg... 
(_tune pid=16924) AMP: checks passed ✅ [repeated 3x across cluster]
(_tune pid=16926) optimizer: 'optimizer=auto' found, ignoring 'lr0=0.009618879805317367' and 'momentum=0.7780874566261266' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
(_tune pid=16926) optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0009767610881903372), 63 bias(decay=0.0)
(_tune pid=16924) Plotting labels to runs/detect/tune/tune_raytune/labels.jpg...  [repeated 3x across cluster]
(_tune pid=16926) TensorBoard: model graph visualization added ✅
(_tune pid=16926) Image sizes 640 train, 640 val
(_tune pid=16926) Using 8 dataloader workers
(_tune pid=16926) Logging results to runs/detect/tune/tune_raytune
(_tune pid=16926) Starting training for 200 epochs...
(_tune pid=16926) 
(_tune pid=16926)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Inst

...

    200/200      2.09G    0.02844     0.6397      1.098         14        640:   0%|          | 0/16 [00:00<?, ?it/s]
    199/200       2.1G    0.02276     0.5018     0.9798         17        640:  75%|███████▌  | 12/16 [00:01<00:00, 11.37it/s] [repeated 13x across cluster]
    200/200       2.1G    0.02262     0.4942     0.9629         14        640:  88%|████████▊ | 14/16 [00:01<00:00, 12.47it/s] [repeated 10x across cluster]


(_tune pid=16925) 
(_tune pid=16925) 200 epochs completed in 0.169 hours.
(_tune pid=16925) Optimizer stripped from runs/detect/tune/tune_raytune/weights/last.pt, 6.3MB
(_tune pid=16925) Optimizer stripped from runs/detect/tune/tune_raytune/weights/best.pt, 6.3MB
(_tune pid=16925) 
(_tune pid=16925) Validating runs/detect/tune/tune_raytune/weights/best.pt...
(_tune pid=16925) Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
(_tune pid=16925) Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.94it/s] [repeated 4x across cluster]
    200/200       2.1G    0.02398     0.5575      1.026         14        640:  12%|█▎        | 2/16 [00:00<00:01, 12.64it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  8.57it/s] [repeated 3x across cluster]


(_tune pid=16925)               negative         36         39      0.875      0.897      0.942      0.604
(_tune pid=16925)               positive         27         27      0.909      0.778      0.875      0.601
(_tune pid=16925)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
(_tune pid=16925)                    all         63         66      0.892      0.838      0.909      0.603 [repeated 2x across cluster]
(_tune pid=16925) Speed: 0.1ms preprocess, 0.6ms inference, 0.0ms loss, 0.6ms postprocess per image
(_tune pid=16925) Results saved to runs/detect/tune/tune_raytune


2024-07-04 03:02:12,796	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/under1/Detect/jeongui/coronal/runs/detect/tune/_tune_2024-07-04_02-51-30' in 0.0177s.
2024-07-04 03:02:12,813	INFO tune.py:1041 -- Total run time: 642.48 seconds (642.42 seconds for the tuning loop).


In [18]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.05488586225769321, 'lrf': 0.7180374727086953, 'momentum': 0.8290500829072247, 'weight_decay': 0.0005448831829968968, 'warmup_epochs': 2.1182739966945237, 'warmup_momentum': 0.6135994074133233, 'box': 0.09876569802728466, 'cls': 3.588737402971903, 'hsv_h': 0.09636627605010294, 'hsv_s': 0.3450973669431999, 'hsv_v': 0.7125525342743981, 'degrees': 23.800271388880702, 'translate': 0.5112401049845391, 'scale': 0.833036974463395, 'shear': 0.7103605819788694, 'perspective': 8.712929970154071e-05, 'flipud': 0.02021839744032572, 'fliplr': 0.832619845547938, 'bgr': 0.7781567509498505, 'mosaic': 0.8700121482468192, 'mixup': 0.978618342232764, 'copy_paste': 0.7991585642167236, 'data': '/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}, Last Reported Metrics: {'metrics/precision(B)': 0.54027, 'metrics/recall(B)': 0.74217, 'metrics/mAP50(B)': 0.6769, 'metrics/mAP50-95(B)': 0.28036, 'val/box_loss': 0.0251, 'v

In [19]:
# Extract the best trial configuration
best_result = result_grid.get_best_result(metric=metric, mode="max")
best_config = best_result.config
print("Best hyperparameters found were: ", best_config)

Best hyperparameters found were:  {'lr0': 0.021046152281773353, 'lrf': 0.1376370346783048, 'momentum': 0.7198627733511899, 'weight_decay': 0.0003637107709426226, 'warmup_epochs': 2.8509838520893984, 'warmup_momentum': 0.4166714377892043, 'box': 0.19790729085066072, 'cls': 0.5877702808425067, 'hsv_h': 0.020887675609483472, 'hsv_s': 0.14517856609649663, 'hsv_v': 0.5877974929188586, 'degrees': 11.398122114290196, 'translate': 0.4196796955706757, 'scale': 0.21998303280144246, 'shear': 1.5896958364551972, 'perspective': 0.00011037514116430514, 'flipud': 0.6563295894652734, 'fliplr': 0.1381829513486138, 'bgr': 0.1965823616800535, 'mosaic': 0.3687251706609641, 'mixup': 0.8209932298479351, 'copy_paste': 0.09710127579306127, 'data': '/home/under1/Detect/jeongui/coronal/coronal_t1wce_2_class.yaml', 'epochs': 200, 'name': 'tune/tune_raytune'}


In [28]:
experiment_name = 'train'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

# 'data' 항목 제거
if 'data' in best_config:
    del best_config['data']
if 'name' in best_config:
    del best_config['name']

# 최고의 하이퍼파라미터로 모델 재훈련
best_model = YOLO("yolov8n.pt")  # 모델 재초기화
best_results = best_model.train(data="coronal_t1wce_2_class.yaml",
                                optimizer='Adam',
                                augment=True,
                                nms=True,
                                name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'),
                                **best_config)

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coronal_t1wce_2_class.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train_raytune3, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sho

train: Scanning /home/under1/Detect/dataset/coronal/labels/train.cache... 255 images, 0 backgrounds, 0 corrupt: 100%|██████████| 255/255 [00:00<?, ?it/s]
val: Scanning /home/under1/Detect/dataset/coronal/labels/val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/train_raytune3/labels.jpg... 
optimizer: Adam(lr=0.021046152281773353, momentum=0.7198627733511899) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0003637107709426226), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/detect/train/train_raytune3
Starting training for 200 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/200      2.27G    0.05142      5.045      1.812         32        640: 100%|██████████| 16/16 [00:01<00:00,  8.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.23it/s]

                   all         63         66          0          0          0          0


...

    200/200      2.11G    0.02623     0.8285      1.218         13        640: 100%|██████████| 16/16 [00:01<00:00, 11.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  9.33it/s]

                   all         63         66      0.716      0.705      0.776      0.453



200 epochs completed in 0.167 hours.
Optimizer stripped from runs/detect/train/train_raytune3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/train_raytune3/weights/best.pt, 6.3MB

Validating runs/detect/train/train_raytune3/weights/best.pt...
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  5.24it/s]


                   all         63         66      0.576      0.713      0.702      0.448
              negative         36         39      0.728      0.686      0.785      0.483
              positive         27         27      0.424      0.741      0.618      0.413
Speed: 0.1ms preprocess, 2.6ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/detect/train/train_raytune3


In [29]:
# train
best_results.results_dict

{'metrics/precision(B)': 0.5760258762809185,
 'metrics/recall(B)': 0.7134231535940937,
 'metrics/mAP50(B)': 0.7017755720527825,
 'metrics/mAP50-95(B)': 0.44792664292950546,
 'fitness': 0.47331153584183316}

In [30]:
# val
experiment_name = 'val'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model = YOLO("/home/under1/Detect/jeongui/coronal/runs/detect/train/train_raytune3/weights/best.pt")
best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'))

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)
Model summary (fused): 168 layers, 3006038 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/under1/Detect/dataset/coronal/labels/val.cache... 63 images, 0 backgrounds, 0 corrupt: 100%|██████████| 63/63 [00:00<?, ?it/s]


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.46it/s]


                   all         63         66      0.761      0.654       0.77      0.491
              negative         36         39      0.888      0.641       0.86      0.522
              positive         27         27      0.634      0.667      0.681      0.459
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/val/val_raytune3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f944c4c9420>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [31]:
# test
experiment_name = 'test'
work_name = 'raytune' #이름수정주의
train_dir = os.path.join('runs', experiment_name, f'{experiment_name}_{work_name}')

if os.path.exists(train_dir):
    shutil.rmtree(train_dir)

best_model.val(name=os.path.join(experiment_name, f'{experiment_name}_{work_name}'), split="test")

Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-2.1.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)


val: Scanning /home/under1/Detect/dataset/coronal/labels/test.cache... 78 images, 0 backgrounds, 0 corrupt: 100%|██████████| 78/78 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.87it/s]


                   all         78         83       0.46      0.719       0.49      0.299
              negative         32         34      0.412      0.618      0.413      0.241
              positive         46         49      0.508      0.821      0.568      0.357
Speed: 0.2ms preprocess, 1.1ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/test/test_raytune3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f942833ace0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804